In [21]:
import pandas as pd
import numpy as np
import os
from functools import reduce

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from rank_bm25 import BM25Okapi
import nltk
from nltk.tokenize import word_tokenize

from openai import OpenAI

import rapidfuzz

# Reading data

In [3]:
excel_file_path = "./fab-employers/Employer Match Data - BCG1.xlsx"
excel_data = pd.read_excel(excel_file_path, sheet_name=0, header=0)

In [4]:
excel_data.columns

Index(['EID Employer Name', 'TML List Names'], dtype='object')

In [5]:
data_column=excel_data.columns[0]
database_column=excel_data.columns[1]
texts_to_match = excel_data[data_column].dropna().unique().tolist()
database_entries = excel_data[database_column].dropna().unique().tolist()
print("Texts to match: ",len(texts_to_match))
print("Database entries: ", len(database_entries))

Texts to match:  9485
Database entries:  13451


# TFIDF

In [6]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=True, norm='l2')
database_tfidf = tfidf_vectorizer.fit_transform(database_entries)
texts_tfidf = tfidf_vectorizer.transform(texts_to_match)
cosine_similarities = cosine_similarity(texts_tfidf, database_tfidf)
top_match_indices = np.argmax(cosine_similarities, axis=1)

In [7]:
top_matches_tfidf = pd.DataFrame({
    'Item': texts_to_match,
    'Matched With': np.array(database_entries)[top_match_indices],
    'Cosine Similarity': cosine_similarities[np.arange(len(texts_to_match)), top_match_indices]
})

In [8]:
top_matches_tfidf

,Item,Matched With,Cosine Similarity
0,UNITED FOODS CO. (PSC),UNITED FOODS CO. (PSC),1.000000
1,PERFECT LIFE PHARMACY,S Z R LIFE PHARMACY (BR OF LIFE PHARMACY L.L.C),0.717237
2,TOTALENERGIES RENEWABLES DEVELOPMENT MIDDLE EA...,TOTALENERGIES RENEWABLES DEVELOPMENT MIDDLE EA...,1.000000
3,AL FAHAD PROJECT MANAGEMENT SERVICES,AL FAHAD PROJECT MANAGEMENT SERVICES,1.000000
4,RSL FREIGHT CO. L.L.C,RSL FREIGHT CO LLC,0.977525
...,...,...,...
9480,SNAP LIVE THEATRICAL SHOW PRODUCTION L.L.C,SNAP LIVE THEATRICAL SHOW PRODUCTION L.L.C,1.000000
9481,HAMPTON BY HILTON MARJAN ISLAND,HAMPTON BY HILTON MARJAN ISLAND L.L.C,1.000000
9482,ROYAL OFFICE GENERAL TRADING L.L.C,ROYAL GULF INTERNATIONAL GENERAL TRADING L.L.C,0.571870
9483,ENJOY CAR RENTAL L.L.C,ENJOY CAR RENTAL L.L.C,1.000000


# BM25

In [9]:
 nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/cegielarafal/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [10]:
tokenized_corpus = [word_tokenize(doc) for doc in database_entries]
bm25 = BM25Okapi(tokenized_corpus)

In [11]:
tokenized_queries = [word_tokenize(text) for text in texts_to_match]
scores_matrix = np.array([bm25.get_scores(query) for query in tokenized_queries])
best_match_indices = np.argmax(scores_matrix, axis=1)

In [12]:
top_matches = [{
    'Item': texts_to_match[i],
    'Matched With': database_entries[best_match_indices[i]],
    'BM25 Score': scores_matrix[i, best_match_indices[i]]
} for i in range(len(texts_to_match))]
top_matches_bm25 = pd.DataFrame(top_matches)

In [13]:
top_matches_bm25

,Item,Matched With,BM25 Score
0,UNITED FOODS CO. (PSC),UNITED FOODS CO. (PSC),26.647880
1,PERFECT LIFE PHARMACY,LIFE GUARD PHARMACY L.L.C,11.602948
2,TOTALENERGIES RENEWABLES DEVELOPMENT MIDDLE EA...,TOTALENERGIES RENEWABLES DEVELOPMENT MIDDLE EA...,32.077268
3,AL FAHAD PROJECT MANAGEMENT SERVICES,AL FAHAD PROJECT MANAGEMENT SERVICES,19.015010
4,RSL FREIGHT CO. L.L.C,RSL FREIGHT CO LLC,17.114946
...,...,...,...
9480,SNAP LIVE THEATRICAL SHOW PRODUCTION L.L.C,SNAP LIVE THEATRICAL SHOW PRODUCTION L.L.C,37.899920
9481,HAMPTON BY HILTON MARJAN ISLAND,HAMPTON BY HILTON MARJAN ISLAND L.L.C,35.632568
9482,ROYAL OFFICE GENERAL TRADING L.L.C,ROYAL SEAGULL GENERAL TRADING L.L.C,10.307209
9483,ENJOY CAR RENTAL L.L.C,ENJOY CAR RENTAL L.L.C,21.865758


# Embeddings

In [14]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [17]:
def get_embeddings(data):
    response = client.embeddings.create(
        input=data, 
        model="text-embedding-ada-002"
    )

    #print(response.data)
    return [e.embedding for e in response.data]

In [18]:
def process_by_chunk(data, process_chunk, chunk_size):
    processed_chunks = []
    
    for start_idx in range(0, len(data), chunk_size):
        print(f"Processing {start_idx}-{start_idx+chunk_size} of {len(data)}", end="\r")
        end_idx = min(start_idx + chunk_size, len(data))
        chunk = data[start_idx:end_idx]
        
        processed_chunk = process_chunk(chunk)
        processed_chunks.append(processed_chunk)

    return np.concatenate(processed_chunks)

In [19]:
text_embeddings = process_by_chunk(texts_to_match, get_embeddings, 1000)

Processing 9000-10000 of 9485

In [20]:
database_embeddings = process_by_chunk(database_entries, get_embeddings, 1000)
#database_embeddings = np.array(get_embeddings(database_entries[:100]))

Processing 13000-14000 of 13451

In [22]:
similarity_scores = cosine_similarity(text_embeddings, database_embeddings)

In [23]:
top_matches = [{
    'Item': texts_to_match[i],
    'Matched With': database_entries[best_match_indices[i]],
    'Similarity Score': similarity_scores[i, best_match_indices[i]]
} for i in range(len(texts_to_match))]
top_matches_embeddings = pd.DataFrame(top_matches)

In [24]:
top_matches_embeddings

,Item,Matched With,Similarity Score
0,UNITED FOODS CO. (PSC),UNITED FOODS CO. (PSC),0.999999
1,PERFECT LIFE PHARMACY,LIFE GUARD PHARMACY L.L.C,0.906440
2,TOTALENERGIES RENEWABLES DEVELOPMENT MIDDLE EA...,TOTALENERGIES RENEWABLES DEVELOPMENT MIDDLE EA...,0.999999
3,AL FAHAD PROJECT MANAGEMENT SERVICES,AL FAHAD PROJECT MANAGEMENT SERVICES,0.999999
4,RSL FREIGHT CO. L.L.C,RSL FREIGHT CO LLC,0.989701
...,...,...,...
9480,SNAP LIVE THEATRICAL SHOW PRODUCTION L.L.C,SNAP LIVE THEATRICAL SHOW PRODUCTION L.L.C,0.999999
9481,HAMPTON BY HILTON MARJAN ISLAND,HAMPTON BY HILTON MARJAN ISLAND L.L.C,0.974100
9482,ROYAL OFFICE GENERAL TRADING L.L.C,ROYAL SEAGULL GENERAL TRADING L.L.C,0.913983
9483,ENJOY CAR RENTAL L.L.C,ENJOY CAR RENTAL L.L.C,0.999997


# Fuzzy

In [25]:
def find_best_fuzzy_match(query, database, threshold=50):
    match_ = rapidfuzz.process.extractOne(query, database, scorer=rapidfuzz.fuzz.ratio, score_cutoff=threshold)
    if match_:
        match, score, index = match_
        return [match, score]
    else:
        return [None, None]

In [26]:
top_matches_fuzzy = pd.DataFrame(columns=['Item', 'Matched With', 'Fuzzy Score'])
top_matches_fuzzy[['Matched With', 'Fuzzy Score']] = [find_best_fuzzy_match(t, database_entries) for t in texts_to_match]
top_matches_fuzzy['Item'] = texts_to_match

In [28]:
top_matches_fuzzy

,Item,Matched With,Fuzzy Score
0,UNITED FOODS CO. (PSC),UNITED FOODS CO. (PSC),100.0
1,PERFECT LIFE PHARMACY,SAFE LIFE PHARMACY L.L.C,71.111111
2,TOTALENERGIES RENEWABLES DEVELOPMENT MIDDLE EA...,TOTALENERGIES RENEWABLES DEVELOPMENT MIDDLE EA...,100.0
3,AL FAHAD PROJECT MANAGEMENT SERVICES,AL FAHAD PROJECT MANAGEMENT SERVICES,100.0
4,RSL FREIGHT CO. L.L.C,RSL FREIGHT CO LLC,92.307692
...,...,...,...
9480,SNAP LIVE THEATRICAL SHOW PRODUCTION L.L.C,SNAP LIVE THEATRICAL SHOW PRODUCTION L.L.C,100.0
9481,HAMPTON BY HILTON MARJAN ISLAND,HAMPTON BY HILTON MARJAN ISLAND L.L.C,91.176471
9482,ROYAL OFFICE GENERAL TRADING L.L.C,ROYAL DOLPHIN GENERAL TRADING L.L.C,86.956522
9483,ENJOY CAR RENTAL L.L.C,ENJOY CAR RENTAL L.L.C,100.0


# Save results

In [34]:
dfs = [top_matches_embeddings, top_matches_tfidf, top_matches_bm25, top_matches_fuzzy]
suffixes = ['emb', 'tfidf', 'bm25', 'fuzzy']
#merged_result = reduce(lambda left, right: pd.merge(left, right, on='Item', how='inner'), dfs)

merged_result = dfs[0]
for i in range(1, len(dfs)):
    merged_result = pd.merge(
        merged_result, 
        dfs[i], 
        on='Item', 
        how='inner',
        suffixes=(suffixes[i-1], suffixes[i])
    )


In [36]:
merged_result

,Item,Matched Withemb,Similarity Score,Matched Withtfidf,Cosine Similarity,Matched Withbm25,BM25 Score,Matched Withfuzzy,Fuzzy Score
0,UNITED FOODS CO. (PSC),UNITED FOODS CO. (PSC),0.999999,UNITED FOODS CO. (PSC),1.000000,UNITED FOODS CO. (PSC),26.647880,UNITED FOODS CO. (PSC),100.0
1,PERFECT LIFE PHARMACY,LIFE GUARD PHARMACY L.L.C,0.906440,S Z R LIFE PHARMACY (BR OF LIFE PHARMACY L.L.C),0.717237,LIFE GUARD PHARMACY L.L.C,11.602948,SAFE LIFE PHARMACY L.L.C,71.111111
2,TOTALENERGIES RENEWABLES DEVELOPMENT MIDDLE EA...,TOTALENERGIES RENEWABLES DEVELOPMENT MIDDLE EA...,0.999999,TOTALENERGIES RENEWABLES DEVELOPMENT MIDDLE EA...,1.000000,TOTALENERGIES RENEWABLES DEVELOPMENT MIDDLE EA...,32.077268,TOTALENERGIES RENEWABLES DEVELOPMENT MIDDLE EA...,100.0
3,AL FAHAD PROJECT MANAGEMENT SERVICES,AL FAHAD PROJECT MANAGEMENT SERVICES,0.999999,AL FAHAD PROJECT MANAGEMENT SERVICES,1.000000,AL FAHAD PROJECT MANAGEMENT SERVICES,19.015010,AL FAHAD PROJECT MANAGEMENT SERVICES,100.0
4,RSL FREIGHT CO. L.L.C,RSL FREIGHT CO LLC,0.989701,RSL FREIGHT CO LLC,0.977525,RSL FREIGHT CO LLC,17.114946,RSL FREIGHT CO LLC,92.307692
...,...,...,...,...,...,...,...,...,...
9480,SNAP LIVE THEATRICAL SHOW PRODUCTION L.L.C,SNAP LIVE THEATRICAL SHOW PRODUCTION L.L.C,0.999999,SNAP LIVE THEATRICAL SHOW PRODUCTION L.L.C,1.000000,SNAP LIVE THEATRICAL SHOW PRODUCTION L.L.C,37.899920,SNAP LIVE THEATRICAL SHOW PRODUCTION L.L.C,100.0
9481,HAMPTON BY HILTON MARJAN ISLAND,HAMPTON BY HILTON MARJAN ISLAND L.L.C,0.974100,HAMPTON BY HILTON MARJAN ISLAND L.L.C,1.000000,HAMPTON BY HILTON MARJAN ISLAND L.L.C,35.632568,HAMPTON BY HILTON MARJAN ISLAND L.L.C,91.176471
9482,ROYAL OFFICE GENERAL TRADING L.L.C,ROYAL SEAGULL GENERAL TRADING L.L.C,0.913983,ROYAL GULF INTERNATIONAL GENERAL TRADING L.L.C,0.571870,ROYAL SEAGULL GENERAL TRADING L.L.C,10.307209,ROYAL DOLPHIN GENERAL TRADING L.L.C,86.956522
9483,ENJOY CAR RENTAL L.L.C,ENJOY CAR RENTAL L.L.C,0.999997,ENJOY CAR RENTAL L.L.C,1.000000,ENJOY CAR RENTAL L.L.C,21.865758,ENJOY CAR RENTAL L.L.C,100.0


In [37]:
merged_result.to_excel('output.xlsx', index=False)